In [1]:
import ast
import csv
import datetime
import gzip
import io
import json
import math
import os
import random
import re
import sys
import xml.etree.ElementTree as ET
from collections import Counter, defaultdict
from typing import Dict, List, Set

from biorgroup.utils.atomic import is_biochemical, is_candidate
import df2img
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import matplotlib.ticker as ticker
import networkx as nx
import numpy as np
import pandas as pd
from rdkit import Chem, RDLogger
from rdkit.Chem import AllChem, Descriptors, Draw, GetPeriodicTable
from rdkit.Chem import rdfiltercatalog
from rdkit.Chem import rdDepictor,rdMolTransforms
from rdkit.Chem import rdRGroupDecomposition as rdRGD
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.EnumerateStereoisomers import EnumerateStereoisomers
from rdkit.Chem.rdRGroupDecomposition import RGroupDecompositionParameters
from rdkit.DataStructs import TanimotoSimilarity
from rdkit.Geometry import Point3D
import seaborn as sns
from tqdm import tqdm

sys.path.append(os.path.join(os.environ['CONDA_PREFIX'],'share','RDKit','Contrib'))
from SA_Score import sascorer
from NP_Score import npscorer

RDLogger.DisableLog("rdApp.*")
csv.field_size_limit(sys.maxsize)
%matplotlib inline

<frozen importlib._bootstrap>:488: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterHierarchyMatcher> already registered; second conversion method ignored.
<frozen importlib._bootstrap>:488: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry> already registered; second conversion method ignored.


In [39]:
# Colors
ATOM_COLORS = {
    0: (102/255.0, 102/255.0, 102/255.0), # dummy
    7: (0.33, 0.6, 0.78), # azote
    8: (0.8, 0.38, 0.33), # oxygen
    15: (0.92, 0.6, 0.3), # phosphore
}

def count_log(x) -> float:
    return math.log(x, 10) if x > 0 else 0

def ticker_formatter(x, pos):
    #value = int(np.round(int(x) / 1000) * 1000)
    return f"{x:,.0f}"

def hex_to_rgb(hex_str):
    hex_str = hex_str.lstrip("#")
    r = int(hex_str[0:2], 16) / 255.0
    g = int(hex_str[2:4], 16) / 255.0
    b = int(hex_str[4:6], 16) / 255.0
    return (r, g, b)

def pick_row(path, chebi_id):
    with gzip.open(path, mode="rt") as csvfile:
        reader = csv.DictReader(csvfile)
        for ix, row in enumerate(reader):
            chebis = ast.literal_eval(row["chebi"])
            if chebi_id in chebis:
                return row

def set_label_dummy(mol: Chem.rdchem.Mol) -> Chem.rdchem.Mol:
    for a in mol.GetAtoms():
        if a.GetAtomicNum() == 0: 
            a.SetProp("atomLabel", "R")
    return mol

def bonds_connected_to_dummies(mol):
    out = []
    for a in mol.GetAtoms():
        if a.GetAtomicNum() == 0:                     # it's a '*'
            aid = a.GetIdx()
            # bonds attached to this dummy atom
            bond_ids = [b.GetIdx() for b in a.GetBonds()]
            # neighbors too (often handy)
            nbr_ids  = [n.GetIdx() for n in a.GetNeighbors()]
            out.append({"dummy_idx": aid,
                        "bond_idxs": bond_ids,
                        "neighbor_atoms": nbr_ids})
    return out

def rotate_180_in_plane(mol, copy_mol=True):
    m = Chem.Mol(mol) if copy_mol else mol
    if m.GetNumConformers() == 0:
        rdDepictor.Compute2DCoords(m, canonOrient=False)

    conf = m.GetConformer(0)
    xs, ys = [], []
    for i in range(m.GetNumAtoms()):
        p = conf.GetAtomPosition(i)
        xs.append(p.x); ys.append(p.y)
    cx = 0.5*(min(xs)+max(xs))
    cy = 0.5*(min(ys)+max(ys))

    T_to = np.array([[1,0,0,-cx],[0,1,0,-cy],[0,0,1,0],[0,0,0,1]], float)
    Rz = np.array([[-1,0,0,0],[0,-1,0,0],[0,0,1,0],[0,0,0,1]], float)  # 180° about Z
    T_bk = np.array([[1,0,0,cx],[0,1,0,cy],[0,0,1,0],[0,0,0,1]], float)

    rdMolTransforms.TransformConformer(conf, T_bk @ Rz @ T_to)
    return m
    
def draw_mol(filename, mol: Chem.rdchem.Mol, core: str = "", rotate: str = ""):
    # Draw
    d2d = Draw.MolDraw2DSVG(1500, 500)    
    opts = d2d.drawOptions()
    opts.updateAtomPalette(ATOM_COLORS)

    if core:
        patt = Chem.MolFromSmarts(core)
        # Match commmon
        hit_ats = list(mol.GetSubstructMatch(patt))
        hit_bonds = []
        for bond in patt.GetBonds():
            aid1 = hit_ats[bond.GetBeginAtomIdx()]
            aid2 = hit_ats[bond.GetEndAtomIdx()]
            hit_bonds.append(mol.GetBondBetweenAtoms(aid1, aid2).GetIdx())
        # Select reverse
        hit_atoms_set = set(hit_ats)
        hit_bonds_set = set(hit_bonds)
        all_atom_ids = set(range(mol.GetNumAtoms()))
        all_bond_ids = set(b.GetIdx() for b in mol.GetBonds())
        # Dummy atoms - bonds
        patt_dummy_qids = [a.GetIdx() for a in patt.GetAtoms() if a.GetAtomicNum() == 0]
        dummy_mol_atoms = {hit_ats[qid] for qid in patt_dummy_qids}
        dumy_connects = bonds_connected_to_dummies(patt)
        dumy_bonds = set([bond for dumy_connect in dumy_connects for bond in dumy_connect["bond_idxs"]])
        # non hit atoms
        non_hit_atoms = sorted((all_atom_ids - hit_atoms_set).union(dummy_mol_atoms))
        non_hit_bonds = sorted((all_bond_ids - hit_bonds_set) - dumy_bonds)
        # Colors
        atom_colors = {at: hex_to_rgb("#b2df8a") for at in non_hit_atoms}
        
    # Build label overrides for dummy atoms
    mol = set_label_dummy(mol=mol)

    if rotate == "canon":
        # https://github.com/rdkit/rdkit/discussions/4939
        rdDepictor.Compute2DCoords(mol, canonOrient=False) #needed to have one 2d "conformer"
    elif rotate == "x":
        rdDepictor.Compute2DCoords(mol, canonOrient=False)
        rot_x = np.array([
            [ -1.,  0.,  0., 0.],
            [ 0., 1.,  0., 0.],
            [ 0.,  0., 1., 0.],
            [ 0.,  0.,  0., 1.],
        ])
        rdMolTransforms.TransformConformer(mol.GetConformer(0), rot_x)
    elif rotate == "y":
        rdDepictor.Compute2DCoords(mol, canonOrient=False)
        rot_y = np.array([
            [-1.,0.,0.,0.],
            [0.,1.,0.,0.],
            [0.,0.,-1.,0.],
            [0.,0.,0.,1.]
        ]) # rotate 180 about y axis
        rdMolTransforms.TransformConformer(mol.GetConformer(0), rot_y)
    elif rotate == "flip":
        rdDepictor.Compute2DCoords(mol, canonOrient=False)
        mol = rotate_180_in_plane(mol)
        
    if core:
        d2d.DrawMolecule(
            mol,
            highlightAtoms=non_hit_atoms,
            highlightBonds=non_hit_bonds,
            highlightAtomColors=atom_colors,
        )
    else:
        d2d.DrawMolecule(mol)
        
    d2d.FinishDrawing()

    # Save - SVG
    svg = d2d.GetDrawingText()
    with open(filename + ".svg", "w", encoding="utf-8") as fd:
        fd.write(svg)

def draw_rxn(filename, rxn=None):
    # Draw
    d2d = Draw.MolDraw2DSVG(1500, 500)
    dopts = d2d.drawOptions()
    dopts.updateAtomPalette(ATOM_COLORS)
    d2d.DrawReaction(rxn)
    d2d.FinishDrawing()
    # Export
    data = d2d.GetDrawingText()
    with open(filename + ".svg", "w") as fd:
        fd.write(data)
        
def compute_weight(smiles) -> float:
    mol = Chem.MolFromSmiles(smiles)
    return Descriptors.MolWt(mol)

def has_isotope(mol) -> bool:
    return any(atom.GetIsotope() != 0 for atom in mol.GetAtoms())

In [3]:
seed = 42
random.seed(a=seed)

# Input
work_dir = os.getcwd()
rgroup_dir = os.path.join(work_dir, "rgroup")
chebi_dir = os.path.join(rgroup_dir, "chebi")

file_biorgroup_csv = os.path.join(chebi_dir, "chebis.csv.gz")
file_rhea_csv = os.path.join(rgroup_dir, "rhea", "rhea-chebi-smiles.csv")
file_metanetx_csv = os.path.join(work_dir, "metanetx", "4.4", "chem_prop.tsv")
file_chembl_csv = os.path.join(work_dir, "chembl", "chembl_35_chemreps.txt.gz")
file_emolecules_csv = os.path.join(work_dir, "emolecules", "emolecules-2025-08-01.txt.gz")

# Output
outdir = os.getcwd()
outdir_image = os.path.join(outdir, "image")
outdir_data = os.path.join(outdir, "data")
os.makedirs(outdir_image, exist_ok=True)
os.makedirs(outdir_data, exist_ok=True)

## Figure - 1 (paper)

**ChEBI candidates**
* 65286 (retained)
* 141495
* 138603 

**Rhea ID containing ChEBI 138603**
* 16801

In [ ]:
# Figure 1 - 1
# indole + L-serine = H20 + L-tryptophan
mol_sub_a = Chem.MolFromSmiles("O=C(*)[C@@H](N*)CC=1C=CC(=CC1)O") # CHEBI:46858
mol_sub_a = set_label_dummy(mol=mol_sub_a)
mol_sub_b = Chem.MolFromSmiles("Nc1ncnc2n(cnc12)[C@@H]1O[C@H](COP([O-])(=O)OS([O-])(=O)=O)[C@@H](OP([O-])([O-])=O)[C@H]1O") # CHEBI:58339
mol_pro_a = Chem.MolFromSmiles("Nc1ncnc2n(cnc12)[C@@H]1O[C@H](COP([O-])([O-])=O)[C@@H](OP([O-])([O-])=O)[C@H]1O") # CHEBI:58343
mol_pro_b = Chem.MolFromSmiles("C=1C=C(C=CC1C[C@@H](C(*)=O)N*)OS([O-])(=O)=O") # CHEBI:65286
mol_pro_b = set_label_dummy(mol=mol_pro_b)
mol_pro_c = Chem.MolFromSmiles("[H+]") # CHEBI:15378

rxn_main = AllChem.ChemicalReaction()
rxn_main.AddReactantTemplate(mol_sub_a)
rxn_main.AddReactantTemplate(mol_sub_b)

rxn_main.AddProductTemplate(mol_pro_a)
rxn_main.AddProductTemplate(mol_pro_b)
rxn_main.AddProductTemplate(mol_pro_c)

draw_rxn(
    filename=os.path.join(outdir_image, "figure.1.rhea_16801"),
    rxn=rxn_main,
)

In [ ]:
# Figure 1 - 2
row = pick_row(path=file_biorgroup_csv, chebi_id="CHEBI:65286")
#row = {'smiles': '*N[C@@H](Cc1ccc(OS(=O)(=O)[O-])cc1)C(*)=O', 'chebi': "['CHEBI:65286']", 'num_heavy_atoms': '16', 'exact_mol_wt': '242.01', 'core_superstructure_smiles': "['COC(=O)[C@@H](N)Cc1ccc(OS(=O)(=O)[O-])cc1', 'CCOC(=O)[C@@H]1Cc2cc(OS(=O)(=O)[O-])c(OS(=O)(=O)[O-])cc2CN1C(=O)c1ccc(OS(=O)(=O)[O-])c(OS(=O)(=O)[O-])c1', 'O=C([O-])[C@@H]1Cc2cc(OS(=O)(=O)[O-])c(OS(=O)(=O)[O-])cc2CN1C(=O)c1ccc(OS(=O)(=O)[O-])c(OS(=O)(=O)[O-])c1', 'O=C([O-])[C@@H]1Cc2cc(OS(=O)(=O)[O-])c(OS(=O)(=O)[O-])cc2CN1C(=O)c1cc(OS(=O)(=O)[O-])c(OS(=O)(=O)[O-])c(OS(=O)(=O)[O-])c1', 'N[C@@H](Cc1ccc(OS(=O)(=O)[O-])cc1)C(=O)[O-]']", 'core_superstructure_pubchem_cid': '[[21145083], [23635706], [44127982], [44127984], [102601667]]', 'rgroup_extended_smiles': "['COC(=O)[C@@H](N)Cc1ccc(OS(=O)(=O)[O-])cc1', 'N[C@@H](Cc1ccc(OS(=O)(=O)[O-])cc1)C(=O)[O-]']", 'rgroup_extended_pubchem_cid': '[[21145083], [102601667]]'}

draw_mol(
    filename=os.path.join(outdir_image, "figure.1.chebi_65286_main"),
    mol=Chem.MolFromSmiles(row["smiles"]),
)
for ix, smiles in enumerate(ast.literal_eval(row["rgroup_extended_smiles"])):
    print("smiles:", smiles)
    draw_mol(
        filename=os.path.join(outdir_image, f"figure.1.chebi_65286_rgroup_extended_{ix}"),
        mol=Chem.MolFromSmiles(smiles),
        core="N[C@@H](Cc1ccc(OS(=O)(=O)[O-])cc1)C(*)=O",
    )

## Figure - 2 (paper)

In [ ]:
# Figure 2 - CHEBI:88242
row = pick_row(path=file_biorgroup_csv, chebi_id="CHEBI:88242")
draw_mol(
    filename=os.path.join(outdir_image, "figure.2.main"),
    mol=Chem.MolFromSmiles(row["smiles"]),
    rotate="flip",
)

for ix, smiles in enumerate(ast.literal_eval(row["core_superstructure_smiles"])):
    draw_mol(
        filename=os.path.join(outdir_image, f"figure.2.core_superstructure_{ix}"), 
        mol=Chem.MolFromSmiles(smiles),
        core=row["smiles"],
        #rotate=True,
    )
                
for ix, smiles in enumerate(ast.literal_eval(row["rgroup_extended_smiles"])):
    draw_mol(
        filename=os.path.join(outdir_image, f"figure.2.rgroup_extended_{ix}"),
        mol=Chem.MolFromSmiles(smiles),
        core=row["smiles"],
        #rotate="canon",
    )

# CHEBI which does not match
for chebi_id in ["CHEBI:142234", "CHEBI:83726"]:
    row = pick_row(path=file_biorgroup_csv, chebi_id=chebi_id)
    chebi_label = chebi_id.lower().replace(":", "_")
    draw_mol(
        filename=os.path.join(outdir_image, f"figure.2.empty_0_{chebi_label}"),
        mol=Chem.MolFromSmiles(row["smiles"]),
    )

## Figure - 3 (paper)

In [ ]:
def compute_sa_score(mol):
    score = sascorer.calculateScore(mol)
    return score

np_model = npscorer.readNPModel()
def compute_np_likeness(mol):
    score = npscorer.scoreMolWConfidence(mol, np_model)
    return (score.nplikeness, score.confidence)

# https://greglandrum.github.io/rdkit-blog/posts/2023-01-09-rgd-tutorial.html#handing-of-chirality
def get_first_stereoisomer(smi):
    # Create a molecule and set options
    mol = Chem.MolFromSmiles(smi)
    options = StereoEnumerationOptions(onlyUnassigned=True, unique=True)
    # Generate stereoisomers (as a generator)
    stereoisomers = EnumerateStereoisomers(mol, options=options)
    # Get only the first stereoisomer
    first_isomer = next(stereoisomers, None)  # None is a default if the generator is empty
    return Chem.MolToSmiles(first_isomer)

def is_potentially_stereogenic_double_bond(bond):
    if bond.GetBondType() != Chem.BondType.DOUBLE:
        return False
    atoms = bond.GetStereoAtoms()
    # If RDKit found potential stereo atoms, then this C=C could have E/Z
    return len(atoms) == 2

def count_ring(smiles: str):
    mol = Chem.MolFromSmiles(smiles)
    return mol.GetRingInfo().NumRings()
    
def analyze_stereo(smiles: str):
    mol = Chem.MolFromSmiles(smiles)    
    # ensure stereo perception and CIP assignment is up to date
    Chem.AssignStereochemistry(mol, cleanIt=True, force=True, flagPossibleStereoCenters=True)
    # ---- Tetrahedral stereocenters (atoms) ----
    chiral_centers = Chem.FindMolChiralCenters(
        mol, includeUnassigned=True, useLegacyImplementation=True
    )

    num_atom_centers_defined = sum(1 for _, lab in chiral_centers if lab in ("R","S"))
    num_atom_centers_undefined = sum(1 for _, lab in chiral_centers if lab == "?")
    num_atom_centers_total = len(chiral_centers)
    
    # ---- Stereogenic double bonds (E/Z) ----
    dbl_info = []
    for b in mol.GetBonds():
        if b.GetBondType() != Chem.BondType.DOUBLE:
            continue

        i, j = b.GetBeginAtomIdx(), b.GetEndAtomIdx()
        stereo = b.GetStereo()  # returns rdchem.BondStereo enum
        # map to human-readable
        if stereo == Chem.BondStereo.STEREOZ:
            label = "Z"
        elif stereo == Chem.BondStereo.STEREOE:
            label = "E"
        elif stereo == Chem.BondStereo.STEREOANY:
            # explicitly marked as “some stereo exists but unspecified”
            label = "?"
        else:
            # STEREONONE: may or may not be a stereogenic C=C; check potential below
            label = None

        # Heuristic: is this C=C *potentially* stereogenic?
        # A double bond is potentially E/Z if both ends have two distinct substituents
        # (excluding terminal =O, =N, cumulated allenes, etc.). RDKit has a helper:
        #possible = Chem.BondStereoPossible(b)
        possible = is_potentially_stereogenic_double_bond(b)
        if label is None:
            # not assigned; if potential exists, treat as undefined
            label = "?" if possible else "-"
            
        # 'E'/'Z'/'?'/'-' (dash = not stereogenic)
        dbl_info.append({
            #"bond_idx": b.GetIdx(),
            #"atoms": (i, j),
            "label": label,                     # 'E'/'Z'/'?'/'-' (dash = not stereogenic)
            "potential": bool(possible)
        })
        
    # Counts for double bonds
    defined_db = sum(1 for d in dbl_info if d["label"] in ("E","Z"))
    undefined_db = sum(1 for d in dbl_info if d["label"] == "?")
    potential_db = sum(1 for d in dbl_info if d["potential"])

    return {
        "atom_centers_total": num_atom_centers_total,
        "atom_centers_defined": num_atom_centers_defined,
        "atom_centers_undefined": num_atom_centers_undefined,
        "double_bonds_potential": potential_db,
        "double_bonds_defined": defined_db,
        "double_bonds_undefined": undefined_db,
    }

### NP-likeness

In [ ]:
file_np_csv = os.path.join(outdir_data, "np_total.csv")
if not os.path.isfile(file_np_csv):
    
    print("Parse RGroup")
    file_np_rgroup_csv = os.path.join(outdir_data, "np_rgroup.csv")
    if not os.path.isfile(file_np_rgroup_csv):
        datas = []
        with gzip.open(file_biorgroup_csv, mode="rt") as csvfile:
            reader = csv.DictReader(csvfile)
            for ix, row in tqdm(enumerate(reader)):
                if not is_candidate(row["smiles"]):
                    continue
                chebis = ast.literal_eval(row["chebi"])
                chebi_id = chebis[0]
                #core_superstructure_smiles = ast.literal_eval(row["core_superstructure_smiles"])
                #score_superstructure_smiles.update(core_superstructure_smiles)
                smiles = row["smiles"]
                mol = Chem.MolFromSmiles(smiles)
                np_likeness, np_conf = compute_np_likeness(mol=mol)
                sa_score = compute_sa_score(mol=mol)
                data = dict(id=chebi_id, origin="rgroup-raw", smiles=smiles, np_likeness=np_likeness, np_conf=np_conf, sa_score=sa_score)
                datas.append(data)
                for smiles in ast.literal_eval(row["rgroup_extended_smiles"]):
                    mol = Chem.MolFromSmiles(smiles)
                    np_likeness, np_conf = compute_np_likeness(mol=mol)
                    sa_score = compute_sa_score(mol=mol)
                    data = dict(id=chebi_id, origin="rgroup-expanded", smiles=smiles, np_likeness=np_likeness, np_conf=np_conf, sa_score=sa_score)
                    datas.append(data)
        df_score_rgroup = pd.DataFrame(datas)
        df_score_rgroup.to_csv(file_np_rgroup_csv, index=False)
    else:
        df_score_rgroup = pd.read_csv(file_np_rgroup_csv)
        df_score_rgroup = df_score_rgroup[df_score_rgroup["origin"] == "rgroup-expanded"]
        
    print("Parse MetaNetX")
    file_np_metanetx_csv = os.path.join(outdir_data, "np_metanetx.csv")
    if not os.path.isfile(file_np_metanetx_csv):
        
        df_metanetx = pd.read_csv(file_metanetx_csv, sep="\t", comment="#", names=["ID", "name", "reference", "formula", "charge", "mass", "InChI", "InChIKey", "SMILES"])
        df_metanetx = df_metanetx[~pd.isna(df_metanetx["SMILES"])]
    
        datas = []
        for _, row in tqdm(df_metanetx.iterrows(), total=df_metanetx.shape[0]):
            smiles = row["SMILES"]
            mol = Chem.MolFromSmiles(smiles)
            if mol:
                np_likeness, np_conf = compute_np_likeness(mol=mol)
                sa_score = compute_sa_score(mol=mol)
                data = dict(id=row["ID"], origin="metanetx", smiles=smiles, np_likeness=np_likeness, np_conf=np_conf, sa_score=sa_score)
                datas.append(data)
        df_score_metanetx = pd.DataFrame(datas)
        df_score_metanetx.to_csv(file_np_metanetx_csv, index=False)
    else:
        df_score_metanetx = pd.read_csv(file_np_metanetx_csv)
    
    print("Parse Chembl")
    file_np_chembl_csv =  os.path.join(outdir_data, "np_chembl.csv")
    if not os.path.isfile(file_np_chembl_csv):
        df_chembl = pd.read_csv(file_chembl_csv, sep="\t")
        datas = []
        for _, row in tqdm(df_chembl.iterrows(), total=df_chembl.shape[0]):
            smiles = row["canonical_smiles"]
            mol = Chem.MolFromSmiles(smiles)
            if mol:
                np_likeness, np_conf = compute_np_likeness(mol=mol)
                sa_score = compute_sa_score(mol=mol)
                data = dict(id=row["chembl_id"], origin="chembl", smiles=smiles, np_likeness=np_likeness, np_conf=np_conf, sa_score=sa_score)
                datas.append(data)
        df_score_chembl = pd.DataFrame(datas)
        df_score_chembl.to_csv(file_np_chembl_csv, index=False)
    else:
        df_score_chembl = pd.read_csv(file_np_chembl_csv)
    
    print("Parse emolecules")
    file_np_emolecules_csv = os.path.join(outdir_data, "np_emolecules.csv")
    if not os.path.isfile(file_np_emolecules_csv):
        df_emol = pd.read_csv(file_emolecules_csv, sep=" ")
        datas = []
        for _, row in tqdm(df_emol.iterrows(), total=df_emol.shape[0]):
            smiles = row["isosmiles"]
            mol = Chem.MolFromSmiles(smiles)
            if mol:
                np_likeness, np_conf = compute_np_likeness(mol=mol)
                sa_score = compute_sa_score(mol=mol)
                data = dict(id=row["version_id"], origin="emolecules", smiles=smiles, np_likeness=np_likeness, np_conf=np_conf, sa_score=sa_score)
                datas.append(data)
        df_score_emolecules = pd.DataFrame(datas)
        df_score_emolecules.to_csv(file_np_emolecules_csv, index=False)
    else:
        df_score_emolecules = pd.read_csv(file_np_emolecules_csv)

    print("Concat")
    print("Metanetx:", df_score_metanetx.shape[0])
    print("Chembl:", df_score_chembl.shape[0])
    print("emolecules:", df_score_emolecules.shape[0])
    print("Rgroup", df_score_rgroup.shape[0])
    df_np = pd.concat([df_score_rgroup, df_score_metanetx, df_score_chembl, df_score_emolecules])
    df_np.to_csv(file_np_csv, index=False)
    del df_score_rgroup, df_score_metanetx, df_score_chembl, df_score_emolecules
else:
    df_np = pd.read_csv(file_np_csv)
    
df_np.head()

### Stereochemistry

In [ ]:
file_stereo_csv = os.path.join(outdir_data, "stereo_total.csv")
if not os.path.isfile(file_stereo_csv):
    
    print("Parse RGroup")
    file_stereo_rgroup_csv = os.path.join(outdir_data, "stereo_rgroup.csv")
    if not os.path.isfile(file_stereo_rgroup_csv):
        datas = []
        with gzip.open(file_biorgroup_csv, mode="rt") as csvfile:
            reader = csv.DictReader(csvfile)
            for ix, row in tqdm(enumerate(reader)):
                if not is_candidate(row["smiles"]):
                    continue
                chebis = ast.literal_eval(row["chebi"])
                chebi_id = chebis[0]
                #core_superstructure_smiles = ast.literal_eval(row["core_superstructure_smiles"])
                #score_superstructure_smiles.update(core_superstructure_smiles)
                smiles = row["smiles"]
                data = dict(id=chebi_id, origin="rgroup-raw", smiles=smiles)
                data.update(analyze_stereo(smiles=smiles))
                data["num_ring"] = count_ring(smiles=smiles)
                datas.append(data)
                for smiles in ast.literal_eval(row["rgroup_extended_smiles"]):
                    data = dict(id=chebi_id, origin="rgroup-expanded", smiles=smiles)
                    data.update(analyze_stereo(smiles=smiles))
                    data["num_ring"] = count_ring(smiles=smiles)
                    datas.append(data)
        df_rgroup = pd.DataFrame(datas)
        df_rgroup.to_csv(file_stereo_rgroup_csv, index=False)
    else:
        df_rgroup = pd.read_csv(file_stereo_rgroup_csv)
    df_rgroup = df_rgroup[df_rgroup["origin"] == "rgroup-expanded"]
    
    print("Parse MetaNetX")
    file_stereo_metanetx_csv = os.path.join(outdir_data, "stereo_metanetx.csv")
    if not os.path.isfile(file_stereo_metanetx_csv):
        df_metanetx = pd.read_csv(file_metanetx_csv, sep="\t", comment="#", names=["ID", "name", "reference", "formula", "charge", "mass", "InChI", "InChIKey", "SMILES"])
        df_metanetx = df_metanetx[~pd.isna(df_metanetx["SMILES"])]
        datas = []
        for _, row in tqdm(df_metanetx.iterrows(), total=df_metanetx.shape[0]):
            smiles = row["SMILES"]
            mol = Chem.MolFromSmiles(smiles)
            if mol:
                data = dict(id=row["ID"], origin="metanetx", smiles=smiles)
                data.update(analyze_stereo(smiles=smiles))
                data["num_ring"] = count_ring(smiles=smiles)
                datas.append(data)
        df_metanetx = pd.DataFrame(datas)
        df_metanetx.to_csv(file_stereo_metanetx_csv, index=False)
    else:
        df_metanetx = pd.read_csv(file_stereo_metanetx_csv)
    
    print("Parse Chembl")
    file_stereo_chembl_csv =  os.path.join(outdir_data, "stereo_chembl.csv")
    if not os.path.isfile(file_stereo_chembl_csv):
        df_chembl = pd.read_csv(file_chembl_csv, sep="\t")
        datas = []
        for _, row in tqdm(df_chembl.iterrows(), total=df_chembl.shape[0]):
            smiles = row["canonical_smiles"]
            mol = Chem.MolFromSmiles(smiles)
            if mol:
                data = dict(id=row["chembl_id"], origin="chembl", smiles=smiles)
                data.update(analyze_stereo(smiles=smiles))
                data["num_ring"] = count_ring(smiles=smiles)
                datas.append(data)
        df_chembl = pd.DataFrame(datas)
        df_chembl.to_csv(file_stereo_chembl_csv, index=False)
    else:
        df_chembl = pd.read_csv(file_stereo_chembl_csv)
    
    print("Parse emolecules")
    file_stereo_emolecules_csv = os.path.join(outdir_data, "stereo_emolecules.csv")
    if not os.path.isfile(file_stereo_emolecules_csv):
        df_emol = pd.read_csv(file_emolecules_csv, sep=" ")
        datas = []
        for _, row in tqdm(df_emol.iterrows(), total=df_emol.shape[0]):
            smiles = row["isosmiles"]
            mol = Chem.MolFromSmiles(smiles)
            if mol:
                data = dict(id=row["version_id"], origin="emolecules", smiles=smiles)
                data.update(analyze_stereo(smiles=smiles))
                data["num_ring"] = count_ring(smiles=smiles)
                datas.append(data)
        df_emolecules = pd.DataFrame(datas)
        df_emolecules.to_csv(file_stereo_emolecules_csv, index=False)
    else:
        df_emolecules = pd.read_csv(file_stereo_emolecules_csv)

    print("Concat")
    df_ster = pd.concat([df_rgroup, df_metanetx, df_chembl, df_emolecules])
    df_ster.to_csv(file_stereo_csv, index=False)
    print("Metanetx:", df_metanetx.shape[0])
    print("Chembl:", df_chembl.shape[0])
    print("emolecules:", df_emolecules.shape[0])
    print("Rgroup", df_rgroup.shape[0])
    del df_rgroup, df_metanetx, df_chembl, df_emolecules
    
else:
    df_ster = pd.read_csv(file_stereo_csv)

df_ster.head()

In [ ]:
print("Melt")
df_ster_chiral = df_ster.melt(
    id_vars=["id", "origin", "smiles"],
    value_vars=["atom_centers_total", "atom_centers_defined", "atom_centers_undefined"],
    var_name="chiral",
    value_name="count"
)
# Clean up "chiral" column to contain just "total"/"defined"
df_ster_chiral["chiral"] = df_ster_chiral["chiral"].str.replace("atom_centers_", "")
df_ster_chiral.drop(columns=["id", "smiles"], inplace=True)

df_ster_stats = df_ster_chiral.groupby(["origin", "chiral"])["count"].mean().to_frame()
df_ster_stats.reset_index(inplace=True)
#df_stats = df_stats.transpose()

print("Stats")
df_ster_stats["count"] = df_ster_stats["count"].apply(lambda x: f"{x:.1f}")
df_ster_stats.to_csv(os.path.join(outdir_data, "stereo_mean.csv"), index=False)
df_ster_stats

In [ ]:
# Example: counts per group
df_ster_plot = df_ster_chiral[df_ster_chiral["chiral"] == "defined"]
df_ster_plot["count"] = df_ster_plot["count"].apply(lambda x: x if x < 9 else 9)

df_ster_counts = (
    df_ster_plot.groupby(["origin", "count"], as_index=False)
           .size()
           .rename(columns={"size": "n"})
)

# percent within each origin (index-aligned)
df_ster_counts["percent"] = 100 * df_ster_counts["n"] / df_ster_counts.groupby("origin")["n"].transform("sum")
df_ster_counts["percent_rev"] = df_ster_counts["percent"].apply(lambda x: f"{100 - x:.1f}")
df_ster_counts.to_csv(os.path.join(outdir_data, "stereo_counts.csv"), index=False)
df_ster_counts.head()

### Figure

In [ ]:
map_labels = {
    "emolecules": "eMolecules",
    "metanetx": "MetaNetX",
    "rgroup-expanded": "BioRGroup",
    "chembl": "ChEMBL",
}

# Plot
sns.set_theme(style="whitegrid")
palette_colors = ["C0", "C1", "C2", "C3"]
formatter = ScalarFormatter(useOffset=False, useLocale=None)
formatter.set_scientific(False)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 7))
fig.subplots_adjust(wspace=0.2, hspace=0.25)

# One
ax1 = sns.barplot(data=df_ster_counts, x="count", y="percent", hue="origin", hue_order=map_labels.keys(), alpha=.7, ax=ax1, palette=palette_colors, legend=True, edgecolor="grey", linewidth=.5)
ax1.margins(y=0.05)   # extra space on both ends
sns.despine(ax=ax1)
ax1.set_ylabel("Percentage\n(%)", rotation=0, labelpad=35)
ax1.set_xlabel("Number of chiral centers", labelpad=10)
ticks = ax1.get_xticks()
labels = [item.get_text() for item in ax1.get_xticklabels()]
labels[-1] = r"$\geq$9"
ax1.set_xticklabels(labels)
ax1.text(-0.28, 1.05, 'A', transform=ax1.transAxes, fontsize=16, fontweight='bold')

# Two
ax2 = sns.kdeplot(df_np, x="np_likeness", hue="origin", hue_order=map_labels.keys(), alpha=.7, ax=ax2, palette=palette_colors, legend=False)
sns.despine(ax=ax2)
ax2.set_ylabel("Density", rotation=0, labelpad=35)
ax2.set_xlabel("Natural product-likeness score")
ax2.text(-0.28, 1.05, 'B', transform=ax2.transAxes, fontsize=16, fontweight='bold')

# Legend
leg = ax1.get_legend()
handles = leg.legend_handles
labels  = [t.get_text() for t in leg.get_texts()]
labels = [map_labels[x] for x in labels]
ax1.legend(handles=handles, labels=labels, frameon=False, title="Source", title_fontproperties={'weight': 'bold'}, bbox_to_anchor=(1.35, 1.05))

plt.tight_layout()
plt.savefig(os.path.join(outdir_image, "figure.3.svg"))
plt.savefig(os.path.join(outdir_image, "figure.3.png"))

plt.show()